In [17]:
!pip install gradio openai-whisper torch ffmpeg zyphra


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 95.1 MB/s eta 0:00:00
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=9104bcb630982103c4e2a8c362fa63e2498f59f1313fc4877e20612b6f0b61ab
  Stored in directory: /root/.cache/pip/wheels/56/30/c5/576bdd729f3bc062d62a551be7fefd6ed2f761901568171e4e
Successfully built ffmpeg
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.3
    Uninstalling numpy-2.2.3:
      Successfully uninstalled numpy-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
zonos 0.1.0.dev0 requires numpy>=2.2.2, but you have numpy 2.0.2 which is incompatible.
thinc 8.2.5 requires numpy<2.0.0,>=1.19.0; python_version >= "3.9", but you have numpy 2.0.2 which is incompa

In [ ]:


# ==================================
# 2) IMPORT LIBRARIES
# ==================================
import gradio as gr
import whisper
import tempfile
from zyphra import ZyphraClient  # Assumes the Zyphra package provides this client

# ==================================
# 3) LOAD WHISPER MODEL
# ==================================
model = whisper.load_model("base")

# ==================================
# 4) DEFINE PROCESSING FUNCTION
# ==================================
def process_media(media_file):
    """
    This function:
      - Transcribes and translates the uploaded audio/video into English using Whisper.
      - Uses ZyphraClient (synchronous) to convert the English text to speech.
      - Returns both the synthesized audio and the English subtitles.
    """
    try:
        # Transcribe and translate the media into English
        result = model.transcribe(media_file, task="translate")
        english_transcription = result["text"]

        # ==================================
        # Zyphra TTS API CALL using ZyphraClient
        # ==================================
        api_key = "zsk-c8741b6d61d76f872442699c84ed180e98f43b2b2cf4ed8f8c8da72c70fcfbb3"
        with ZyphraClient(api_key=api_key) as client:
            # Get audio bytes for the given text; adjust speaking_rate if desired.
            audio_data = client.audio.speech.create(
                text=english_transcription,
                speaking_rate=15
            )

        # Write the returned audio data to a temporary file
        temp_audio = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        temp_audio.write(audio_data)
        temp_audio.close()
        synthesized_audio = temp_audio.name

        return synthesized_audio, english_transcription

    except Exception as e:
        print("Error during processing:", e)
        return None, f"Error: {str(e)}"

# ==================================
# 5) BUILD GRADIO INTERFACE
# ==================================
interface = gr.Interface(
    fn=process_media,
    inputs=gr.File(label="Upload Audio or Video", file_types=["audio", "video"]),
    outputs=[
        gr.Audio(type="filepath", label="Synthesized English Audio"),
        gr.Textbox(label="English Subtitles")
    ],
    title="Multilingual Media to English TTS Pipeline (Zyphra)",
    description=(
        "Upload an audio or video file in any language. The file is transcribed and translated into "
        "English using Whisper, then converted to speech via the Zyphra TTS service using ZyphraClient."
    )
)

# ==================================
# 6) LAUNCH THE APP
# ==================================
interface.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9da5e93e59442e1e14.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
